In [1]:
import tensorflow as tf
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
from matplotlib import pyplot as plt
import cv2
import pandas as pd

In [185]:
interpreter = tf.lite.Interpreter(model_path='tf model/lite-model_movenet_singlepose_thunder_3.tflite')
interpreter.allocate_tensors()

In [186]:
shaped = np.squeeze(np.multiply(interpreter.get_tensor(interpreter.get_output_details()[0]['index']), [480,640,1]))

## Variable definition

In [187]:
body_parts = {0: 'nose',
              1: 'left eye',
              2: 'right eye',
              3: 'left ear',
              4: 'right ear',
              5: 'left shoulder',
              6: 'right shoulder',
              7: 'left elbow',
              8: 'right elbow',
              9: 'left wrist',
              10: 'right wrist',
              11: 'left hip',
              12: 'right hip',
              13: 'left knee',
              14: 'right knee',
              15: 'left ankle',
              16: 'right ankle'}




In [188]:
# Define variables for fall detection
NUM_FRAMES_TO_COMPARE = 2
min_time_between_frames = 0.5  # in seconds
max_time_between_frames = 1.5  # in seconds
confidence_threshold = 0.4
angle_threshold = 60.0  # in degrees

In [189]:
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

## Functions

In [190]:
def find_middle_point(point1, point2):
    return (int((point1[0] + point2[0]) / 2), int((point1[1] + point2[1]) / 2))

In [191]:
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    for i in range(17):
        kp = shaped[i]
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 4, (0,255,0), -1)
            # give detail index of keypoint
            # cv2.putText(frame, str(body_parts[i]), (int(kx), int(ky)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 1, cv2.LINE_AA)
    

In [192]:
for edge, color in EDGES.items():
    p1, p2 = edge
    y1, x1, c1 = shaped[p1]
    y2, x2, c2 = shaped[p2]

def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)

In [193]:
import math 
def calculate_angle(keypoints):
    nose = keypoints[0][0], keypoints[0][1]
    hip = find_middle_point(keypoints[11], keypoints[12])

    angle_nose_hip = math.degrees(math.atan2(nose[1] - hip[1], nose[0] - hip[0]))
    angle = math.degrees(angle_nose_hip)
    
    if angle_nose_hip < 0:
        angle = -angle

    return angle

In [194]:
def is_body_line_motion_downward(angle, previous_angles, threshold, num_previous_angles):

    # avg_previous_angles = sum(previous_angles) / len(previous_angles)
    if angle < previous_angles - threshold:
        return True, previous_angles
    else:
        return False, previous_angles

# Live Prediction

In [198]:
 with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    cap = cv2.VideoCapture('video/cam7.avi')
    angles = []
    fall = False
    while cap.isOpened():
        ret, frame = cap.read()
        fall_detected = False
        # Reshape image
        img = frame.copy()
        img = tf.image.resize_with_pad(np.expand_dims(img, axis=0), 256,256)
        input_image = tf.cast(img, dtype=tf.float32)
        
        # Setup input and output 
        input_details = interpreter.get_input_details()
        output_details = interpreter.get_output_details()
        
        # Make predictions 
        interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
        interpreter.invoke()
        keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
        
        y, x, c = frame.shape
        shaped = np.squeeze(np.multiply(keypoints_with_scores, [y,x,1]))
        angle = calculate_angle(shaped)
        angles.append(angle)
        
        
        # Rendering 
        draw_connections(frame, keypoints_with_scores, EDGES, 0.4)
        draw_keypoints(frame, keypoints_with_scores, 0.4)

        nose = shaped[0]
        hip_left = shaped[11]
        hip_right = shaped[12]
        if (hip_right[2]>0.4) and (hip_left[2]>0.4) and (nose[2]>0.4):
            print(angle)
            fall_detected, previous_angles = is_body_line_motion_downward(
                angle=angle,
                previous_angles=angles[-5],
                threshold=60,  # adjust threshold as needed
                num_previous_angles=5  # adjust number of previous angles to keep track of as needed
            )
            print(fall_detected)
            if fall_detected:
                fall = True
            
        if fall:
            cv2.putText(frame, "Fall detected!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 1, cv2.LINE_AA)
        
        cv2.imshow('MoveNet Lightning', frame)
        
        if cv2.waitKey(10) & 0xFF==ord('q'):
            break
            
    cap.release()
    cv2.destroyAllWindows()

8434.318457689285
True
7610.160499217422
True
7975.106523157051
False
9278.420609749024
False
9580.413018810666
False
9396.571676093792
False
9686.711081314725
True
8711.65835806619
True
8239.559063010282
True
8380.406615690375
False
8222.82325400909
True
8730.990227830363
False
8905.13514740914
True
8589.049255157179
True
8864.681790479783
False
8639.384764645853
False
8739.61924664868
False
8902.159536735247
True
9236.197290796812
False
9034.620677347646
False
8977.555684108642
False
8848.141814508934
False
9057.705460727366
False
9291.901780160115
False
9330.500844702934
True
9035.789185194171
True
8934.390606208819
True
8812.799199194009
True
9221.776977881002
True
9327.137837587781
False
9528.585265067804
False
9606.78441955909
False
9342.20869572185
False
8984.337835304212
True
8776.40971492872
True
9348.91089649638
True
9942.912355817454
False
8181.356454132406
True


In [199]:
!pip install mediapipe opencv-python


     ---------------------------------------- 50.2/50.2 MB 1.6 MB/s eta 0:00:00
  Using cached opencv_python-4.7.0.72-cp37-abi3-win_amd64.whl (38.2 MB)
     -------------------------------------- 199.7/199.7 kB 1.7 MB/s eta 0:00:00
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.12
    Uninstalling flatbuffers-1.12:
      Successfully uninstalled flatbuffers-1.12


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\ASUS\\anaconda3\\Lib\\site-packages\\cv2\\cv2.pyd'
Consider using the `--user` option or check the permissions.

